In [1]:
def generic_dp(h, dn, imax, xmax, N, direction, wn): #1
    
    Fn_1 = [x * h for x in range(imax + 1)]
    Wn = [wn(x) for x in range(xmax + 1)]
    
    strategy = [[-1] * (imax + 1) for _ in range(N)]
    
    for n in range (N):
        
        Fn = [float('inf') for _ in range(imax + 1)]
        
        for i in range(len(Wn)):
            for j in range(len(Fn_1)):
                
                inext = j - i + dn
                if direction == 1:
                    inext = j + i - dn
                    
                if 0 <= inext <= imax:
                    if (Wn[i] + Fn_1[j] < Fn[inext]):
                        Fn[inext] = Wn[i] + Fn_1[j]
                        
                        strategy[n][inext] = i
                            
        if n == N - 1:
            
            Fn_1 = [Fn[x] for x in range(len(Fn))]
            
            print('The resulting values are: ' + str(Fn_1))
            if direction:
                print('This is for the end stock values from 0 to ' + str(imax) + '\n')
            else:
                print('This is for the start stock values from 0 to ' + str(imax) + '\n')
            
            return strategy
            
        Fn_1 = [x * h + Fn[x] for x in range(len(Fn))]
        
        print('After ' + str(n + 1) + ' step(s) values for F are the following: ' + str(Fn_1))
    return None
  
def get_optimal_strategy(strategy, i_end, N, dn, direction):
    optimal_production = []
    current_inventory = i_end
    
    for n in reversed(range(N)):
        production = strategy[n][current_inventory]
        optimal_production.append(production)
        
        if (direction == 1):
            current_inventory = current_inventory - production + dn
        else:
            current_inventory = current_inventory + production - dn
    
    if direction:
        optimal_production.reverse()
    return optimal_production

def dynamic_programming(h, dn, imax, xmax, N, direction, wn):
    if (direction):
        print('From the start, for the end values.')
    else:
        print('From the end, for the start values.')
    strategy = generic_dp(h, dn, imax, xmax, N, direction, wn)
    for i in range(imax + 1):
        print('For ' + str(i) + ' stock value: ' + str(get_optimal_strategy(strategy, i, N, dn, direction)))
    print()

dynamic_programming(1, 2, 4, 3, 4, 0, lambda x: 0 if x == 0 else 10 + 3 * x)
dynamic_programming(1, 2, 4, 3, 4, 1, lambda x: 0 if x == 0 else 10 + 3 * x)

From the end, for the start values.
After 1 step(s) values for F are the following: [16, 14, 2, 4, 6]
After 2 step(s) values for F are the following: [32, 22, 18, 17, 6]
After 3 step(s) values for F are the following: [41, 38, 34, 25, 22]
The resulting values are: [57, 53, 41, 38, 34]
This is for the start stock values from 0 to 4

For 0 stock value: [2, 3, 3, 0]
For 1 stock value: [3, 0, 2, 2]
For 2 stock value: [0, 3, 3, 0]
For 3 stock value: [0, 3, 0, 2]
For 4 stock value: [0, 0, 2, 2]

From the start, for the end values.
After 1 step(s) values for F are the following: [2, 4, 6, 20, 24]
After 2 step(s) values for F are the following: [6, 20, 24, 28, 43]
After 3 step(s) values for F are the following: [22, 26, 41, 46, 51]
The resulting values are: [38, 41, 45, 60, 65]
This is for the end stock values from 0 to 4

For 0 stock value: [0, 0, 2, 2]
For 1 stock value: [0, 0, 2, 3]
For 2 stock value: [0, 0, 3, 3]
For 3 stock value: [0, 1, 3, 3]
For 4 stock value: [0, 2, 3, 3]



In [2]:
import numpy as np #2

def get_p(P, p, n):
    for i in range (n):
        p = p @ P
        print('Vector p(' + str(i + 1) + ') is: ' + str(p))
    return p

def get_P(P, n):
    result = P
    for i in range (n - 1):
        result = result @ P
        print('Matrix P(' + str(i + 2) + ') is:\n' + str(result))
    return result

P = np.array([[0.1, 0.6, 0.3],   
              [0.1, 0.8, 0.1],
              [0.2, 0.7, 0.1]])
p = np.array([0,1,0])

p3 = get_p(P, p, 3)
P2 = get_P(P, 2)

Vector p(1) is: [0.1 0.8 0.1]
Vector p(2) is: [0.11 0.77 0.12]
Vector p(3) is: [0.112 0.766 0.122]
Matrix P(2) is:
[[0.13 0.75 0.12]
 [0.11 0.77 0.12]
 [0.11 0.75 0.14]]


In [3]:
def get_q(P, R): #3
    q = [0.0 for _ in range(len(P))]
    for i in range (len(P)):
        for j in range(len(P[i])):
            q[i] += P[i][j] * R[i][j]
    return q

def get_v(P, R, n):
    q = get_q(P, R)
    v = [[0.0] * len(P) for _ in range(n)]
    v[0] = q
    for i in range(1, len(v)):
        for j in range(len(P)):
            v[i][j] = q[j]
            for k in range(len(P[j])):
                v[i][j] += P[j][k] * v[i-1][k]
    return v

P = np.array([[0.2, 0.2, 0.6],   
              [0.2, 0.1, 0.7],
              [0.3, 0.5, 0.2]])
R = np.array([[24.6, 20.1, 19.5],   
              [29.0, 28.5, 20.0],
              [16.0, 15.8, 11.5]])
print('The average 1-step revenue vector is: ' + str(get_q(P, R)))
print('For steps from 1 to 3 the revenues are (for each starting state):\n' + str(get_v(P, R, 3)))

The average 1-step revenue vector is: [20.64, 22.65, 15.0]
For steps from 1 to 3 the revenues are (for each starting state):
[[20.64, 22.65, 15.0], [38.298, 39.543, 35.516999999999996], [57.5184, 59.1258, 53.3643]]


In [4]:
def get_kq(P, R): #4
    q = [[0.0] * len(P[k]) for k in range(len(P))]
    for k in range (len(P)):
        for i in range (len(P[k])):
            for j in range(len(P[k][i])):
                q[k][i] += P[k][i][j] * R[k][i][j]
    return q

def get_kv(P, R, N):
    q = np.array(get_kq(P, R)).transpose().tolist()
    v = [[0.0] * len(P) for _ in range(N)]
    strategy = [[0] * N for _ in range(len(P[0]))]
    v[0] = [max(q[i]) for i in range(len(q))]
    for n in range(1, N):
        qk = v[n-1]
        for i in range(len(P[0])):
            kv = [0 for _ in range(len(P))]
            for k in range(len(P)):
                value = q[i][k]
                for j in range(len(P[k][i])):
                    value += P[k][i][j] * v[n-1][j]
                kv[k] = value
            v[n][i] = max(kv)
            strategy[i][0] = q[i].index(max(q[i]))
            strategy[i][n] = kv.index(max(kv))
    return v, strategy

P = np.array([[[0.5, 0.5],   
               [0.4, 0.6]],
              [[0.8, 0.2],   
               [0.9, 0.1]]])
R = np.array([[[10.0, 4.0],   
               [3.0, -6.0]],
              [[5.0, 5.0],   
               [3.0, -11.0]]])

print('These are the qi for rows=strategies, columns=states: ' + str(get_kq(P, R)))
v, strategy = get_kv(P, R, 3)
print('The results for each step starting with 1 are:\n' + str(v))
print('The strategies for states rows=states, columns=steps:\n' + str(strategy))

These are the qi for rows=strategies, columns=states: [[7.0, -2.3999999999999995], [5.0, 1.6]]
The results for each step starting with 1 are:
[[7.0, 1.6], [11.3, 8.06], [16.68, 12.576]]
The strategies for states rows=states, columns=steps:
[[0, 0, 0], [1, 1, 1]]


In [5]:
def get_p(l12, l21): #5
    p2 = 1 / (1 + l21 / l12)
    return 1 - p2, p2

print(get_p(8, 6))

(0.4285714285714286, 0.5714285714285714)


In [6]:
def get_p(P, p, n): #6
    for i in range (n):
        p = p @ P
    return p

def get_matrices(P):
    absorbingI, absorbingJ = [], []
    for i in range(len(P)):
        for j in range(len(P[i])):
            if P[i][j] == 1:
                absorbingI.append(i)
                absorbingJ.append(j)
    R, Q = [], []
    for i in range(len(P)):
        r, q = [], []
        for j in range(len(P[i])):
            if i not in absorbingI and j not in absorbingJ:
                q.append(P[i][j])
            if i not in absorbingI and j in absorbingJ:
                r.append(P[i][j])
        if r:
            R.append(r)
        if q:
            Q.append(q)
    return R, Q

def get_N(P):
    Q = get_matrices(P)[1]
    return np.linalg.inv(np.subtract(np.eye(len(Q)), Q))

P = np.array([[0.1, 0.5, 0.4],   
              [0.01, 0.54, 0.45],
              [0.0, 0.0, 1.0]])
p = np.array([1.0, 0.0, 0.0])

p2, p3 = get_p(P, p, 2), get_p(P, p, 3) #6.1 - 6.2
print('1. Starting from the state 1, for 2026, the transition probability to state 3 will be the following: ' + str(p2[2]))
print('2. Starting from the state 1, in 3 years, the transition probability to state 3 will be the following: ' + str(p3[2]))

R, Q = get_matrices(P)
N = get_N(P)
n = N @ [[1.0],[1.0]]
print('\n3. From state 1, the average amount of years in state 2 is: ' + str(N[0][1])) #6.3
print('\n4. ' + str(N[0][1]) + ' < ' + str(N[1][1]) + ' which means, that state 2 would appear more times' +
     ' (average) if it is the starting state') #6.4
print('\n5. From state 1, the average amount of years before absorption is: ' + str(n[0])) #6.5
print('\n6. ' + str(n[0]) + ' > ' + str(n[1]) + ' therefore, starting in state 1 usually leads to bigger ' +
     'amount of years before absorption') #6.6

B = N @ R
print('\n7. ' + str(B) + ' while state 3 is the only one absorbing, it is logical that the answer ' +
     'would be 100% for both transitive states') #6.7

1. Starting from the state 1, for 2026, the transition probability to state 3 will be the following: 0.665
2. Starting from the state 1, in 3 years, the transition probability to state 3 will be the following: 0.8150000000000001

3. From state 1, the average amount of years in state 2 is: 1.2224938875305624

4. 1.2224938875305624 < 2.2004889975550124 which means, that state 2 would appear more times (average) if it is the starting state

5. From state 1, the average amount of years before absorption is: [2.34718826]

6. [2.34718826] > [2.22493888] therefore, starting in state 1 usually leads to bigger amount of years before absorption

7. [[1.]
 [1.]] while state 3 is the only one absorbing, it is logical that the answer would be 100% for both transitive states
